In [2]:
%pip install holoviews

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install networkx

In [4]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [5]:
import re
import requests
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts
from bs4 import BeautifulSoup
from urllib.request import urljoin



In [17]:
import requests
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts
from bs4 import BeautifulSoup
from urllib.request import urljoin


In [6]:
sites = "https://www.google.com/" #@param {type:"string"}
depth = 3 #@param {type:"slider", min:1, max:10, step:1}
max_child = 10 #@param {type:"slider", min:1, max:100, step:1}

def crawl(sites, depth=1, current_depth=0, max_child=10):
    get_links = lambda site: pd.Series(urljoin(site, tag.get('href')) for tag in BeautifulSoup(requests.get(site).text).find_all('a')).drop_duplicates().head(max_child)
    
    sites = pd.concat([sites,pd.DataFrame([{'link': link, 'parent': site, 'depth': current_depth + 1}
                                       for site in sites[sites.depth == current_depth].link
                                       for link in get_links(site)])], 
                         ignore_index=True).drop_duplicates('link')
    
    if depth > current_depth + 1:
        return crawl(sites, depth=depth, current_depth=current_depth+1, max_child=max_child)
    else:
        return sites


result = crawl(sites=pd.DataFrame([{'link': site, 'depth': 0, 'parent': site} for site in sites.split()]), 
               max_child=max_child,
               depth=depth)

In [7]:
result

,link,depth,parent
0,https://www.google.com/,0,https://www.google.com/
1,https://www.google.com/imghp?hl=km&tab=wi,1,https://www.google.com/
2,https://maps.google.com.kh/maps?hl=km&tab=wl,1,https://www.google.com/
3,https://play.google.com/?hl=km&tab=w8,1,https://www.google.com/
4,https://news.google.com/?tab=wn,1,https://www.google.com/
...,...,...,...
487,https://support.google.com/websearch/?hl=en,3,https://support.google.com/websearch/?p=displa...
488,https://support.google.com/websearch/community...,3,https://support.google.com/websearch/?p=displa...
491,https://support.google.com/websearch/?p=result...,3,https://support.google.com/websearch/?p=result...
495,https://accounts.google.com/ServiceLogin?hl=en...,3,https://support.google.com/websearch/?p=result...


In [44]:
# this is just the example scenarios
# from the results you can see the links "https://support.google.com/websearch", "https://play.google.com"
# it could be the all services google site

In [8]:
#Task : write the regular expression to get all the services of google
#text = "Google offers various services like Google Search, Google Maps, Google Drive, and Google Photos, Google Play."

list_of_link = []
for index, row in result.iterrows():
    list_of_link.append(row['link'])
#     print(link)
item = []
item2 = []
list_of_service = []
for a in range(len(list_of_link)):
    item = list_of_link[a].split("://")
    item2 = item[1].split(".")[0]
    list_of_service.append(item2)
#     print(list_of_link[a])
#     print(item2)
    
print((set(list_of_service)))
print(len(set(list_of_service)))

{'console', 'maps', 'admin', 'classroom', 'policies', 'blog', 'workspace', 'drive', 'learning', 'play', 'mail', 'crisisresponse', 'sustainability', 'www', 'news', 'accounts', 'trends', 'support', 'calendar', 'myaccount', 'safety'}
21


In [11]:
#Task : Visualize the services : - how many services that google have? 

# Make a request to Google Discovery API
response = requests.get("https://www.googleapis.com/discovery/v1/apis")

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    services = data.get('items', [])
    total_services = len(services)
    print(f"Google has {total_services} services.")
else:
    print("Failed to get the list of services.")

Google has 426 services.
